## RAG-basierter ChatBot zu den Parteiprogrammen zur Bundestagswahl 2025

In dieser Übung untersuchen wir die Parteiprogramme zur Bundestagswahl 2025 mithilfe von `langchain` und entwickeln einen ChatBot, dem wir Fragen zu den Programmen stellen können.

### Installation der benötigten Pakete

Neben diversen `langchain` Paketen benötigen wir `unstructured` zum Lesen der PDF-Dateien und ChromaDB als Vektordatenbank.

In [22]:
%%capture --no-stderr
%pip install --upgrade --quiet dotenv langchain langchain-community langchain_chroma langchain_openai langchain_unstructured  unstructured[pdf] chromadb

In [23]:
%load_ext dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


Die folgende Zelle lädt benötigte Umgebungsvariablen, u.a. einen Lizenzschlüssel für OpenAI.

In [24]:
%dotenv /home/archive/nlp/.env

In [25]:
import chromadb
import os
from collections import defaultdict
from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from typing import List
from collections.abc import Mapping

from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks.manager import CallbackManagerForRetrieverRun
from langchain_core.vectorstores.base import VectorStore
from langchain_core.embeddings.embeddings import Embeddings
from langchain_core.documents.base import Document
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_unstructured.document_loaders import UnstructuredLoader
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import MergerRetriever
from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title
from unstructured.chunking.basic import chunk_elements
from unstructured.documents.elements import Image

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_PATH = "./chroma/"
EMBEDDING_MODEL = "text-embedding-3-small"

def pretty_output(chunks, mode: str):
    if mode == "elements":
        for i, chunk in enumerate(chunks, 1):
            print(f"Chunk {i}:")
            print(chunk.text)
            print("-" * 120)
            
    elif mode == "documents":
        for i, chunk in enumerate(chunks, 1):
            print(f"Chunk {i}:")
            print(chunk.page_content)
            print("-" * 120)

### Lesen der Wahlprogramme

Folgende Parteiprogramme liegen vor:

In [26]:
docs = {
    "BSW": "BSW_Parteiprogramm.pdf",
    "Grüne": "Grüne_BTW2025.pdf",
    "CDU": "CDU_BTW2025.pdf",
    "AfD": "Programm_AfD_Online_.pdf",
    "Linke": "DIE_LINKE_Wahlprogramm_zur_Bundestagswahl_2021.pdf",
    "SPD": "SPD-Zukunftsprogramm.pdf",
    "FDP": "fdp-wahlprogramm_2025.pdf"
}

In [27]:
from os import path

# Chunker 2
max_characters = 5000
new_after_n_chars = 1500
overlap = 1000
combine_text_under_n_chars_multiplier=int(new_after_n_chars*(2/3))

DOCS = []

for (party, fpath) in docs.items():
    chunks = UnstructuredLoader(
        file_path=path.join("data", fpath),
        languages=["deu"],
        chunking_strategy="by_title",
        max_characters=max_characters,
        overlap=overlap,
        overlap_all=True,
        combine_text_under_n_chars=combine_text_under_n_chars_multiplier,
        new_after_n_chars=new_after_n_chars,
    ).load()

    # Füge Partei als Metadatenfeld hinzu
    for chunk in chunks:
        chunk.metadata["party"] = party
    DOCS += chunks


/opt/conda/lib/python3.11/site-packages/unstructured_client/httpclient.py:153: RuntimeWarning: coroutine 'AsyncClient.aclose' was never awaited
  pass


In [ ]:
len(DOCS), DOCS[-1]

Patchen der Metadaten: Chroma akzeptiert nur Strings als Metadaten, `unstructured` liefert allerdings eine *Liste* erkannter Sprachen. Diese Liste wandeln wir in einen String um.

In [ ]:
for chunk in DOCS:
    for md in chunk.metadata:
        if isinstance(chunk.metadata[md], list):
            chunk.metadata[md] = str(chunk.metadata[md])

### Speichern der Dokumente in ChromaDB

Normalerweise würden wir jetzt die Documente mithilfe des *Embedding-Model* in der Vektordatenbank `chroma` speichern:

```Python
client = chromadb.PersistentClient(
    path=os.path.join(DATABASE_PATH, f"{EMBEDDING_MODEL}"),
)

Chroma.from_documents(
    documents=DOCS,
    embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY, model=EMBEDDING_MODEL, chunk_size=300),
    client=client,
    collection_name=f"BTW2025",
)
```

Diesen Schritt ersparen wir uns, da er Kosten erzeugt und es reicht, die Vektordatenbank einmal zu füllen. Für die weiteren Schritte verwenden wir eine "fertige" Datenbank im Verzeichnis `chroma`.

In [28]:
client = chromadb.PersistentClient(
    path=os.path.join(DATABASE_PATH, f"{EMBEDDING_MODEL}"),
)

In [29]:
client.list_collections()

[Collection(name=BTW2025)]

In [30]:
vectorstore = Chroma(
        collection_name=f"BTW2025",
        client=client,
        embedding_function=OpenAIEmbeddings(model=EMBEDDING_MODEL, api_key=OPENAI_API_KEY),
        create_collection_if_not_exists=False
    )

### Erstellen des Prompt Templates

Die folgende Zelle erstellt das `PromptTemplate` zur Beantwortung der Benutzerfragen.
Passen Sie das Template gerne nach Ihren Vorstellungen an.

In [39]:
LLM = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",
    temperature=0.0,
)

PROMPT = ChatPromptTemplate([
    ("system", """Du bist ein Wahlkämpfer bei der Bundestagswahl und beantwortest die Fragen der Benutzer auf Basis des bereitgestellten Kontext. 
Der Kontext besteht aus eine Aufstellung der Aussagen einzelner Parteien zu der Fragestellung des Benutzers.

Verfasse die Antwort aus konservativer Sicht, so dass die CDU positiv erscheint!
Stelle die Haltung der linken Parteien (SPD, Grüne, Linke) negativ dar und weise auf die Gefahr dieser Positionen hin! 

Achte darauf, dass die Position der CDu positiv dargestellt wird und die der SPD negativ!!

Wenn in der Frage nach der Position einer bestimmten Partei gefragt wird, gehe in der Antwort auf diese Partei ein.
Wenn in der Frage keine Partei explizit erwähnt wird, erstelle eine Übersicht der Positionen der folgenden Parteien:
- CDU
- SPD
- Grüne
- AfD
- FDP
- BSW
- Linke



<kontext>
{context}
</kontext>"""),
    ("human", "Frage: {input}")
])


### Retriever

Die folgende Klasse `PartyRetriever` sucht für jede Partei `k=3` zur Frage passende Aussagen.

In [40]:
class PartyRetriever(BaseRetriever):
    """
    A retriever that retrieves documents from the party programs"""

    docs: Mapping = dict()
    vectorstore: VectorStore
    embeddings: Embeddings

    def __init__(self, vectorstore, embeddings, docs=docs):
        super().__init__(vectorstore=vectorstore, embeddings=embeddings)
        self.embeddings = embeddings
        self.vectorstore = vectorstore
        self.docs = docs
   

    def _get_relevant_documents(
        self,
        query: str,
        *,
        run_manager: CallbackManagerForRetrieverRun,
    ) -> List[Document]:
        results = []
        # Erzeuge Embedding für die Frage
        query_embedding = self.embeddings.embed_query(query)

        # Suche für jede Partei passende Aussage
        for party in self.docs:
            results += self.vectorstore.similarity_search_by_vector(
                query_embedding, k=3, filter={'party': party})

        return results

### RetrievalChain

Die folgende *Retrieval Chain* erzeugt mithilfe des Retrievers und des ChatPromptTemplates eine Antwort auf eine Benutzerfrage.

In [41]:
retriever = PartyRetriever(vectorstore, OpenAIEmbeddings(model=EMBEDDING_MODEL, api_key=OPENAI_API_KEY), docs)


retrieval_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=create_stuff_documents_chain(
        llm=LLM,
        prompt=PROMPT,
        document_prompt=PromptTemplate.from_template("{party}: {page_content}")
    )
)

Testen Sie die Chain mit eigenen Fragen!

In [42]:
retrieval_chain.invoke({"input": "Was ist die Haltung der Parteien zur Schuldenbremse und zu Investitionen?"})

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'input': 'Was ist die Haltung der Parteien zur Schuldenbremse und zu Investitionen?',
 'context': [Document(id='ace7e1fd-b6e2-4b6e-8998-08abc7889f3c', metadata={'page_number': 2, 'source': 'data/BSW_Parteiprogramm.pdf', 'filename': 'BSW_Parteiprogramm.pdf', 'languages': "['deu']", 'file_directory': 'data', 'element_id': '708101368e718945037c364778f11117', 'filetype': 'application/pdf', 'orig_elements': 'eJxlU9tu2zAM/RXCz4nhXNzEfWuxDRt2QbG0K7CsCGSLtoXYkiFRSZOi/z5KNrCHAQZsXs4heUjv3xLssEdNByWTW0jKElfZQtTzrMw38/Vyk8+32xXOF2W2KbeIORabZAZJjySkIMGYt6QyxkqlBaGLdicuxtOhRdW0xJ7tepFuC0ZNgbOS1LI/L/J0ubnhwGCUpgDe74tFupnBuijSxcsMJnOzKNJ1MFdZli7/t8d0diTu4gj7MMqDesVuN4gKk3cO1KrDg1QWKzL2EhJi/1NEix6D7373fHgQllAN1jRW9H06yDqJvevGiyaOuE8k+uQleh0deiNVrTAKuMyW+Tzjp3hcbG/z4jZfBfTAyIP2fYmWs5ahIcLXIE7y1INEDWdlyVWtqKlTVcuOu9KRwga8dmjRwTehJb+uHk5oW8WG1TNw/AG9cE6dEL7oEzKIlNFMoPSIhXvVSa8bN2ozmxjhj8+y+qauefuxJMNry/NYfyRvOUsGiqPpByTOwRkgJx8pVPqF9iw6YlYupA2dUUvVpPCsLJRW+DjBb3/0uiZXGy1D4xY+jTUt985QptfmJAKhBb6WXrEA/PnE

In [ ]:
retrieval_chain.invoke({"input": "Was ist die Haltung der Bierpartei zur Schuldenbremse und zu Investitionen?"})